# Raspberry Pi 4
> [Main Table of Contents](../README.md)

## In This Notebook
> References: [Raspberry Pi Docs](https://www.raspberrypi.com/documentation/computers/remote-access.html#setting-up-an-ssh-server)  
- Enable Raspberry Pi OS SSH
- SSH headless entry into Raspberry Pi OS

## Enable Raspberry Pi OS SSH
- Rasp Pi Menu dropdown -> Preferences -> Raspberry Pi Configuration -> Interfaces -> Toggle SSH

## SSH headless entry into Raspberry Pi OS using password (passwordless using keys is not setup)
- Get Host IP.  Terminal on Raspberry Pi
    ```
    hostname -I
    ```
- Terminal on another device
    - Username & password is that for Raspberry Pi and configured when first set up the Raspberry Pi OS
        ```
        ssh <username>@<host ip>
        ```
